In [1]:
-- 20-1. Creating an AFTER DML Trigger

USE AdventureWorks2014;

Commands completed successfully.

Total execution time: 00:00:00.056

In [2]:
-- Create a table to Track all Inserts and Deletes
CREATE TABLE Production.ProductInventoryAudit
    (
    ProductID INT NOT NULL,
    LocationID SMALLINT NOT NULL,
    Shelf NVARCHAR(10) NOT NULL,
    Bin TINYINT NOT NULL,
    Quantity SMALLINT NOT NULL,
    rowguid UNIQUEIDENTIFIER NOT NULL,
    ModifiedDate DATETIME NOT NULL,
    InsertOrDelete CHAR(1) NOT NULL
    );
GO
-- Create trigger to populate Production.ProductInventoryAudit table
CREATE TRIGGER Production.trg_id_ProductInventoryAudit ON Production.ProductInventory
    AFTER INSERT, DELETE
AS
BEGIN
    SET NOCOUNT ON;
-- Inserted rows
    INSERT Production.ProductInventoryAudit
        (ProductID,
        LocationID,
        Shelf,
        Bin,
        Quantity,
        rowguid,
        ModifiedDate,
        InsertOrDelete)
        SELECT DISTINCT
            i.ProductID,
            i.LocationID,
            i.Shelf,
            i.Bin,
            i.Quantity,
            i.rowguid,
            GETDATE(),
            'I'
        FROM inserted i
        UNION ALL
        SELECT d.ProductID,
            d.LocationID,
            d.Shelf,
            d.Bin,
            d.Quantity,
            d.rowguid,
            GETDATE(),
            'D'
        FROM deleted d;
END
GO
-- Insert a new row
INSERT Production.ProductInventory
    (ProductID,
    LocationID,
    Shelf,
    Bin,
    Quantity)
VALUES (316,
    6,
    'A',
    4,
    22);
-- Delete a row
DELETE Production.ProductInventory
WHERE ProductID = 316
    AND LocationID = 6;
-- Check the audit table
SELECT ProductID,
    LocationID,
    InsertOrDelete
FROM Production.ProductInventoryAudit;

: Msg 2714, Level 16, State 6, Line 2
There is already an object named 'ProductInventoryAudit' in the database.

: Msg 2714, Level 16, State 2, Procedure trg_id_ProductInventoryAudit, Line 2
There is already an object named 'trg_id_ProductInventoryAudit' in the database.

Shelf 'A' is closed for new inventory.

: Msg 3609, Level 16, State 1, Line 53
The transaction ended in the trigger. The batch has been aborted.

Total execution time: 00:00:00.062

In [3]:
-- 20-2. Creating an INSTEAD OF DML Trigger

-- Create Department "Approval" table
CREATE TABLE HumanResources.DepartmentApproval
    (
    Name NVARCHAR(50) NOT NULL
        UNIQUE,
    GroupName NVARCHAR(50) NOT NULL,
    ModifiedDate DATETIME NOT NULL
        DEFAULT GETDATE()
    ) ;
GO
-- Create view to see both approved and pending approval departments
CREATE VIEW HumanResources.vw_Department
AS
    SELECT Name,
        GroupName,
        ModifiedDate,
        'Approved' Status
    FROM HumanResources.Department
    UNION
    SELECT Name,
        GroupName,
        ModifiedDate,
        'Pending Approval' Status
    FROM HumanResources.DepartmentApproval ;
GO
-- Create an INSTEAD OF trigger on the new view
CREATE TRIGGER HumanResources.trg_vw_Department ON HumanResources.vw_Department
    INSTEAD OF INSERT
AS
    SET NOCOUNT ON
    INSERT HumanResources.DepartmentApproval
        (Name,
        GroupName)
        SELECT i.Name,
            i.GroupName
        FROM inserted i
        WHERE i.Name NOT IN (
            SELECT Name
            FROM HumanResources.DepartmentApproval) ;
GO
-- Insert into the new view, even though view is a UNION
-- of two different tables
INSERT HumanResources.vw_Department
    (Name,
    GroupName)
VALUES ('Print Production',
    'Manufacturing') ;
-- Check the view's contents
SELECT Status,
    Name
FROM HumanResources.vw_Department
WHERE GroupName = 'Manufacturing' ;    

: Msg 2714, Level 16, State 6, Line 4
There is already an object named 'DepartmentApproval' in the database.

: Msg 2714, Level 16, State 3, Procedure vw_Department, Line 2
There is already an object named 'vw_Department' in the database.

: Msg 2111, Level 16, State 1, Procedure trg_vw_Department, Line 2
Cannot create trigger 'trg_vw_Department' on view 'HumanResources.vw_Department' because an INSTEAD OF INSERT trigger already exists on this object.

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.116

Status,Name
Approved,Production
Approved,Production Control
Pending Approval,Print Production


In [4]:
-- 20-3. Handling Transactions in Triggers

ALTER TRIGGER Production.trg_id_ProductInventoryAudit ON Production.ProductInventory
    AFTER INSERT, DELETE
AS
    SET NOCOUNT ON ;
    IF EXISTS ( SELECT Shelf
        FROM inserted
        WHERE Shelf = 'A' )
    BEGIN
        PRINT 'Shelf ''A'' is closed for new inventory.' ;
        ROLLBACK ;
    END
-- Inserted rows
    INSERT Production.ProductInventoryAudit
        (ProductID,
        LocationID,
        Shelf,
        Bin,
        Quantity,
        rowguid,
        ModifiedDate,
        InsertOrDelete)
        SELECT DISTINCT
            i.ProductID,
            i.LocationID,
            i.Shelf,
            i.Bin,
            i.Quantity,
            i.rowguid,
            GETDATE(),
            'I'
        FROM inserted i ;
-- Deleted rows
    INSERT Production.ProductInventoryAudit
        (ProductID,
        LocationID,
        Shelf,
        Bin,
        Quantity,
        rowguid,
        ModifiedDate,
        InsertOrDelete)
        SELECT d.ProductID,
            d.LocationID,
            d.Shelf,
            d.Bin,
            d.Quantity,
            d.rowguid,
            GETDATE(),
            'D'
        FROM deleted d;        
    IF EXISTS ( SELECT Quantity
        FROM deleted
        WHERE Quantity > 0 )
    BEGIN
        PRINT 'You cannot remove positive quantity rows!' ;
        ROLLBACK ;
    END
GO

Commands completed successfully.

Total execution time: 00:00:00.007

In [5]:
INSERT Production.ProductInventory
    (ProductID,
    LocationID,
    Shelf,
    Bin,
    Quantity)
VALUES (316,
    6,
    'A',
    4,
    22) ;

Shelf 'A' is closed for new inventory.

: Msg 3609, Level 16, State 1, Line 1
The transaction ended in the trigger. The batch has been aborted.

Total execution time: 00:00:00.033

In [6]:
BEGIN TRANSACTION ;
-- Deleting a row with a zero quantity
DELETE Production.ProductInventory
WHERE ProductID = 853
    AND LocationID = 7 ;
-- Deleting a row with a non-zero quantity
DELETE Production.ProductInventory
WHERE ProductID = 999
    AND LocationID = 60 ;
COMMIT TRANSACTION ;

(1 row affected)

You cannot remove positive quantity rows!

: Msg 3609, Level 16, State 1, Line 7
The transaction ended in the trigger. The batch has been aborted.

Total execution time: 00:00:00.525

In [7]:
SELECT ProductID,
    LocationID
FROM Production.ProductInventory
WHERE (ProductID = 853
        AND LocationID = 7)
    OR (ProductID = 999
        AND LocationID = 60) ;

(2 rows affected)

Total execution time: 00:00:00.022

ProductID,LocationID
853,7
999,60


In [8]:
-- 20-4. Linking Trigger Execution to Modified Columns

CREATE TRIGGER HumanResources.trg_U_Department ON HumanResources.Department
    AFTER UPDATE
AS
    IF UPDATE(GroupName)
        BEGIN
            PRINT 'Updates to GroupName require DBA involvement.' ;
            ROLLBACK ;
        END
GO

Commands completed successfully.

Total execution time: 00:00:00.009

In [9]:
-- 20-5. Viewing DML Trigger Metadata

-- Show the DML triggers in the current database
SELECT OBJECT_NAME(parent_id) Table_or_ViewNM,
    name TriggerNM,
    is_instead_of_trigger,
    is_disabled
FROM sys.triggers
WHERE parent_class_desc = 'OBJECT_OR_COLUMN'
ORDER BY OBJECT_NAME(parent_id),
    Name ;

(13 rows affected)

Total execution time: 00:00:00.676

Table_or_ViewNM,TriggerNM,is_instead_of_trigger,is_disabled
Department,trg_U_Department,0,0
Employee,dEmployee,1,0
Person,iuPerson,0,0
ProductInventory,trg_id_ProductInventoryAudit,0,0
PurchaseOrderDetail,iPurchaseOrderDetail,0,0
PurchaseOrderDetail,uPurchaseOrderDetail,0,0
PurchaseOrderHeader,uPurchaseOrderHeader,0,0
SalesOrderDetail,iduSalesOrderDetail,0,0
SalesOrderHeader,uSalesOrderHeader,0,0
Vendor,dVendor,1,0


In [10]:
-- Displays the trigger SQL definition --(if the trigger is not encrypted)
SELECT o.name,
    m.definition
FROM sys.sql_modules m
    INNER JOIN sys.objects o
        ON m.object_id = o.object_id
WHERE o.type = 'TR'
    AND o.name = 'trg_id_ProductInventoryAudit'

(1 row affected)

Total execution time: 00:00:00.064

name,definition
trg_id_ProductInventoryAudit,"-- 20-3. Handling Transactions in Triggers CREATE TRIGGER Production.trg_id_ProductInventoryAudit ON Production.ProductInventory AFTER INSERT, DELETE AS SET NOCOUNT ON ; IF EXISTS ( SELECT Shelf FROM inserted WHERE Shelf = 'A' ) BEGIN PRINT 'Shelf ''A'' is closed for new inventory.' ; ROLLBACK ; END -- Inserted rows INSERT Production.ProductInventoryAudit (ProductID, LocationID, Shelf, Bin, Quantity, rowguid, ModifiedDate, InsertOrDelete) SELECT DISTINCT i.ProductID, i.LocationID, i.Shelf, i.Bin, i.Quantity, i.rowguid, GETDATE(), 'I' FROM inserted i ; -- Deleted rows INSERT Production.ProductInventoryAudit (ProductID, LocationID, Shelf, Bin, Quantity, rowguid, ModifiedDate, InsertOrDelete) SELECT d.ProductID, d.LocationID, d.Shelf, d.Bin, d.Quantity, d.rowguid, GETDATE(), 'D' FROM deleted d; IF EXISTS ( SELECT Quantity FROM deleted WHERE Quantity > 0 ) BEGIN PRINT 'You cannot remove positive quantity rows!' ; ROLLBACK ; END"


In [11]:
-- 20-6. Creating a DDL Trigger

CREATE TABLE dbo.DDLAudit
    (
    EventData XML NOT NULL,
    AttemptDate DATETIME NOT NULL
    DEFAULT GETDATE(),
    DBUser CHAR(50) NOT NULL
    ) ;
GO

: Msg 2714, Level 16, State 6, Line 3
There is already an object named 'DDLAudit' in the database.

Total execution time: 00:00:00.007

In [12]:
CREATE TRIGGER db_trg_INDEXChanges ON DATABASE
    FOR CREATE_INDEX, ALTER_INDEX, DROP_INDEX
AS
    SET NOCOUNT ON ;
    INSERT dbo.DDLAudit
        (EventData, DBUser)
    VALUES (EVENTDATA(), USER) ;
GO

Commands completed successfully.

Total execution time: 00:00:00.090

In [13]:
CREATE NONCLUSTERED INDEX ni_DDLAudit_DBUser ON
dbo.DDLAudit(DBUser) ;
GO

: Msg 1913, Level 16, State 1, Line 1
The operation failed because an index or statistics with name 'ni_DDLAudit_DBUser' already exists on table 'dbo.DDLAudit'.

Total execution time: 00:00:00.003

In [14]:
SELECT EventData
FROM dbo.DDLAudit

(1 row affected)

Total execution time: 00:00:00.035

EventData
"<EVENT_INSTANCE><EventType>CREATE_INDEX</EventType><PostTime>2020-12-26T16:23:27.593</PostTime><SPID>66</SPID><ServerName>HILL</ServerName><LoginName>HILL\frank</LoginName><UserName>dbo</UserName><DatabaseName>AdventureWorks2014</DatabaseName><SchemaName>dbo</SchemaName><ObjectName>ni_DDLAudit_DBUser</ObjectName><ObjectType>INDEX</ObjectType><TargetObjectName>DDLAudit</TargetObjectName><TargetObjectType>TABLE</TargetObjectType><TSQLCommand><SetOptions ANSI_NULLS=""ON"" ANSI_NULL_DEFAULT=""ON"" ANSI_PADDING=""ON"" QUOTED_IDENTIFIER=""ON"" ENCRYPTED=""FALSE"" /><CommandText>CREATE NONCLUSTERED INDEX ni_DDLAudit_DBUser ON dbo.DDLAudit(DBUser)</CommandText></TSQLCommand></EVENT_INSTANCE>"


In [15]:
-- 20-7. Creating a Logon Trigger

CREATE LOGIN nightworker WITH PASSWORD = 'pass@word1' ;
GO

: Msg 15025, Level 16, State 1, Line 3
The server principal 'nightworker' already exists.

Total execution time: 00:00:00.008

In [16]:
CREATE DATABASE ExampleAuditDB ;
GO
USE ExampleAuditDB ;
GO
CREATE TABLE dbo.RestrictedLogonAttempt
    (
    LoginNM SYSNAME NOT NULL,
    AttemptDT DATETIME NOT NULL
    ) ;
GO

: Msg 1801, Level 16, State 3, Line 1
Database 'ExampleAuditDB' already exists. Choose a different database name.

Commands completed successfully.

: Msg 2714, Level 16, State 6, Line 5
There is already an object named 'RestrictedLogonAttempt' in the database.

Total execution time: 00:00:00.019

In [17]:
USE master ;
GO

Commands completed successfully.

Total execution time: 00:00:00.002

In [18]:
CREATE TRIGGER trg_logon_attempt ON ALL SERVER
    WITH EXECUTE AS 'sa'
        FOR LOGON
AS
    BEGIN
        IF ORIGINAL_LOGIN() = 'nightworker'
            AND DATEPART(hh, GETDATE()) BETWEEN 7 AND 18
            BEGIN
                ROLLBACK ;
                INSERT ExampleAuditDB.dbo.RestrictedLogonAttempt
                    (LoginNM, AttemptDT)
                VALUES (ORIGINAL_LOGIN(), GETDATE()) ;
            END
    END
GO

: Msg 2714, Level 16, State 2, Procedure trg_logon_attempt, Line 1
There is already an object named 'trg_logon_attempt' in the database.

Total execution time: 00:00:00.009

In [19]:
-- 20-8. Viewing DDL Trigger Metadata

-- Show the DML triggers in the current database
SELECT name TriggerNM,
    is_disabled
FROM sys.triggers
WHERE parent_class_desc = 'DATABASE'
ORDER BY OBJECT_NAME(parent_id),
    Name ;

(0 rows affected)

Total execution time: 00:00:00.025

TriggerNM,is_disabled


In [20]:
SELECT name,
    s.type_desc S0L_or_CLR,
    is_disabled,
    e.type_desc FiringEvents
FROM sys.server_triggers s
    INNER JOIN sys.server_trigger_events e
        ON s.object_id = e.object_id ;

(1 row affected)

Total execution time: 00:00:00.052

name,S0L_or_CLR,is_disabled,FiringEvents
trg_logon_attempt,SQL_TRIGGER,0,LOGON


In [21]:
SELECT t.name,
    m.Definition
FROM sys.triggers AS t
    INNER JOIN sys.sql_modules m
        ON t.object_id = m.object_id
WHERE t.parent_class_desc = 'DATABASE' ;

(0 rows affected)

Total execution time: 00:00:00.021

name,Definition


In [22]:
SELECT t.name,
    m.definition
FROM sys.server_sql_modules m
    INNER JOIN sys.server_triggers t
        ON m.object_id = t.object_id ;

(1 row affected)

Total execution time: 00:00:00.039

name,definition
trg_logon_attempt,"CREATE TRIGGER trg_logon_attempt ON ALL SERVER WITH EXECUTE AS 'sa' FOR LOGON AS BEGIN IF ORIGINAL_LOGIN() = 'nightworker' AND DATEPART(hh, GETDATE()) BETWEEN 7 AND 18 BEGIN --ROLLBACK ; INSERT ExampleAuditDB.dbo.RestrictedLogonAttempt (LoginNM, AttemptDT) VALUES (ORIGINAL_LOGIN(), GETDATE()) ; END END"


In [23]:
-- 20-9. Modifying a Trigger

USE master ;
GO
ALTER TRIGGER trg_logon_attempt ON ALL SERVER
    WITH EXECUTE AS 'sa'
        FOR LOGON
AS
    BEGIN
        IF ORIGINAL_LOGIN() = 'nightworker'
            AND DATEPART(hh, GETDATE()) BETWEEN 7 AND 18
            BEGIN
                --ROLLBACK ;
                INSERT ExampleAuditDB.dbo.RestrictedLogonAttempt
                    (LoginNM, AttemptDT)
                VALUES (ORIGINAL_LOGIN(), GETDATE()) ;
            END
    END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [24]:
SELECT LoginNM,
    AttemptDT
FROM ExampleAuditDB.dbo.RestrictedLogonAttempt ;

(0 rows affected)

Total execution time: 00:00:00.017

LoginNM,AttemptDT


In [25]:
-- 20-10. Enabling and Disabling a Trigger

USE AdventureWorks2014;

DISABLE TRIGGER HumanResources.trg_U_Department
ON HumanResources.Department;

Commands completed successfully.

Total execution time: 00:00:00.010

In [26]:
INSERT HumanResources.Department
    (Name,
    GroupName)
VALUES ('Construction',
    'Building Services') ;

: Msg 2601, Level 14, State 1, Line 1
Cannot insert duplicate key row in object 'HumanResources.Department' with unique index 'AK_Department_Name'. The duplicate key value is (Construction).

The statement has been terminated.

Total execution time: 00:00:00.011

In [27]:
INSERT HumanResources.Department
    (Name, GroupName)
VALUES ('Cleaning', 'Building Services') ;

: Msg 2601, Level 14, State 1, Line 1
Cannot insert duplicate key row in object 'HumanResources.Department' with unique index 'AK_Department_Name'. The duplicate key value is (Cleaning).

The statement has been terminated.

Total execution time: 00:00:00.003

In [28]:
-- 20-11. Nesting Triggers

USE master ;
GO
-- Disable nesting
EXEC sp_configure 'nested triggers', 0 ;
RECONFIGURE WITH OVERRIDE ;
GO
-- Enable nesting
EXEC sp_configure 'nested triggers', 1 ;
RECONFIGURE WITH OVERRIDE ;
GO

Commands completed successfully.

Configuration option 'nested triggers' changed from 1 to 0. Run the RECONFIGURE statement to install.

Configuration option 'nested triggers' changed from 0 to 1. Run the RECONFIGURE statement to install.

Total execution time: 00:00:00.068

In [29]:
-- 20-12. Controlling Recursion

-- Allow recursion
ALTER DATABASE AdventureWorks2014
SET RECURSIVE_TRIGGERS ON ;

-- View the db setting
SELECT is_recursive_triggers_on
FROM sys.databases
WHERE name = 'AdventureWorks2012' ;

-- Prevents recursion
ALTER DATABASE AdventureWorks2012
SET RECURSIVE_TRIGGERS OFF ;

-- View the db setting
SELECT is_recursive_triggers_on
FROM sys.databases
WHERE name = 'AdventureWorks2012' ;

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.118

is_recursive_triggers_on


is_recursive_triggers_on


: Msg 5011, Level 14, State 5, Line 13
User does not have permission to alter database 'AdventureWorks2012', the database does not exist, or the database is not in a state that allows access checks.

: Msg 5069, Level 16, State 1, Line 13
ALTER DATABASE statement failed.

In [30]:
-- 20-13. Specifying the Firing Order

CREATE TABLE dbo.TestTriggerOrder (TestID INT NOT NULL) ;
GO
CREATE TRIGGER dbo.trg_i_TestTriggerOrder ON dbo.TestTriggerOrder
    AFTER INSERT
AS
    PRINT 'I will be fired first.' ;
GO

CREATE TRIGGER dbo.trg_i_TestTriggerOrder2 ON dbo.TestTriggerOrder
    AFTER INSERT
AS
    PRINT 'I will be fired last.' ;
GO

CREATE TRIGGER dbo.trg_i_TestTriggerOrder3 ON dbo.TestTriggerOrder
    AFTER INSERT
AS
    PRINT 'I will be somewhere in the middle.' ;
GO

EXEC sp_settriggerorder 'trg_i_TestTriggerOrder', 'First', 'INSERT' ;
EXEC sp_settriggerorder 'trg_i_TestTriggerOrder2', 'Last', 'INSERT' ;

INSERT dbo.TestTriggerOrder
    (TestID)
VALUES (1) ;

: Msg 2714, Level 16, State 6, Line 3
There is already an object named 'TestTriggerOrder' in the database.

: Msg 2714, Level 16, State 2, Procedure trg_i_TestTriggerOrder, Line 1
There is already an object named 'trg_i_TestTriggerOrder' in the database.

: Msg 2714, Level 16, State 2, Procedure trg_i_TestTriggerOrder2, Line 2
There is already an object named 'trg_i_TestTriggerOrder2' in the database.

: Msg 2714, Level 16, State 2, Procedure trg_i_TestTriggerOrder3, Line 2
There is already an object named 'trg_i_TestTriggerOrder3' in the database.

I will be fired first.

I will be somewhere in the middle.

I will be fired last.

(1 row affected)

Total execution time: 00:00:00.118

In [31]:
-- 20-14. Dropping a Trigger

-- Switch context back to the AdventureWorks2012 database
USE AdventureWorks2014 ;
GO
-- Drop a DML trigger
DROP TRIGGER dbo.trg_i_TestTriggerOrder ;
-- Drop multiple DML triggers
DROP TRIGGER dbo.trg_i_TestTriggerOrder2, dbo.trg_i_TestTriggerOrder3 ;
-- Drop a DDL trigger
DROP TRIGGER db_trg_INDEXChanges
ON DATABASE ;

Commands completed successfully.

: Msg 3701, Level 11, State 5, Line 7
Cannot drop the trigger 'dbo.trg_i_TestTriggerOrder', because it does not exist or you do not have permission.

: Msg 3701, Level 11, State 5, Line 9
Cannot drop the trigger 'dbo.trg_i_TestTriggerOrder2', because it does not exist or you do not have permission.

: Msg 3701, Level 11, State 5, Line 9
Cannot drop the trigger 'dbo.trg_i_TestTriggerOrder3', because it does not exist or you do not have permission.

Total execution time: 00:00:00.018